

<span style="color:red">**Team members / emails**</span> --> * BIG DATA*
- (1) MELLAL Houdaifa / hodaifa.mellal@gmail.com 


# 1.Initialisation de l'environnement de travail:

Dans cette partie, on a initialisé l'envrionnement spark avec la version 3.0.2 et hadoop 3.2, une installation de tout les package utilisé a également été fait.

In [ ]:
##mise en place de l'environnement de travail 
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz  
# unzip it
!tar xf spark-3.0.2-bin-hadoop3.2.tgz 
# install findspark 
!pip install -q findspark
!pip install fastavro
!pip install pyarrow

Dans cette partie, on a importé et intialiser l'envrionnement avec spark

In [ ]:
# Set up required environment variables

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.0.1 pyspark-shell'

findspark.init("spark-3.0.2-bin-hadoop3.2")

Dans cette partie, on a lancé spark, en utilisant SQLContext, SPARKContext et SparkSession:

In [ ]:
##Initation de Spark:

from pyspark.sql import SQLContext # For the connexion with spark
from pyspark import SparkContext # For the connexion with spark
from pyspark.sql import SparkSession

sc = SparkContext('local', 'Spark SQL') 
sqlc = SQLContext(sc)
spark = SparkSession.builder.appName('BIGDATA').getOrCreate()

Téléchargement des données qu'on a utilisé durant le projet, il sera stocké dans le repertoire local du notebook. Le fichié téléchargé été compréssé, donc on a utilisé des commandes SHELL afin de lde décompresser:

In [ ]:
!wget -q  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
!tar -xf 20news-19997.tar.gz  

La création des RDDs demandé alt.atheism et rec.sport.baseball, ceci est fait a partir des fichiers text, on a utilisé wholeTextFiles:

In [ ]:
#Create the RDDs from alt.atheism and rec.sport.baseball files:

RDD1 = sc.wholeTextFiles("20_newsgroups/alt.atheism/*")
RDD2= sc.wholeTextFiles("20_newsgroups/rec.sport.baseball/*")

La fonction extract_info est la fonction responsable de explorer les fichiers qui sont stocké dans des RDD, et puis recupèrer uniquement les informations qu'on a besoin, et qu'on va utiliser:

In [ ]:
#The function that extract the informations:

def extract_info(splited_elm):
  i = 0
 
  #Search for the end of header:
  while( i< len(splited_elm) and not splited_elm[i].startswith('Lines:') ):
    i+=1
  if i == len(splited_elm):
    return None
  #Decompose the mail into header and body:
  header = splited_elm[0:i+1]
  body   = '\n'.join(splited_elm[i+1:]).strip()
  header_dict = {'Category':None,'Subject':None, 'Source' : None, 'Organization' : None, 'Lines':None,'Date':None}
  
  #Extract the informations from the header:
  for header_elm in header:

    if (header_elm.lower().startswith('newsgroups')):
      header_dict['Category'] = header_elm[11:].strip()
    
    elif (header_elm.lower().startswith('subject')):
      header_dict['Subject'] = header_elm[9:].strip()

    elif (header_elm.lower().startswith('from')):
      header_dict['Source'] = header_elm[5:].strip()
    
    elif (header_elm.lower().startswith('organization')):
      header_dict['Organization'] = header_elm[13:].strip()
    
    elif (header_elm.lower().startswith('lines')):
      header_dict['Lines'] = header_elm[6:].strip()

    elif (header_elm.lower().startswith('date')):
      header_dict['Date'] = header_elm[5:].strip()

  return (header_dict,body)


A partir des RDD, et a l'aide de la fonction extract_info on a créer des RDDS avec uniquement les informations qu'on va utiliser, on a également fait un union entre les deux RDD et on a supprimer les lignes qui contiennent des valeurs NULL:

In [ ]:
# Apply the traitement to the RDDs:

RDD1 = RDD1.map(lambda x : extract_info(x[1].split('\n')) )
RDD2 = RDD2.map(lambda x : extract_info(x[1].split('\n')) )

# Merge the RDDs Objects:

RDD = RDD1.union(RDD2) 

# Delete the None lines:

RDD = RDD.filter(lambda x : x is not None)

Transformation de chaque des RDD en ROW format, ceci nous facilite la transformation en DataFrame:

In [ ]:
#Tranform the document to Row format:
from pyspark.sql import Row
RDD = RDD.map(lambda x : Row(**x[0],Body = x[1]) )

Transformation de la RDD qui regroupe les articiles des deux sujets en dataframe:

In [ ]:
#Create a DataFrame object from the RDD object
Df = spark.createDataFrame(RDD)

sauvgarde de la DataFrame sous format Avro et Parquet:

In [ ]:
# Save data on Avro and Parquet Formats: 
Df.write.format("avro").save("Emailss.avro")
Df.write.parquet("Emailss.parquet")

<h1>Analyse descriptive</h1>


Dans cette partie, on a explorer la dataframe, ceci afin de faciliter la préparation des données pour la phase de l'application de Kmeans:

Affichage des nombre des organisations:

In [ ]:
#Get the number of the different organizations: 
nbr_org = Df.select("Organization").distinct().count()
print("the number of the distinct organizations is : {}".format(nbr_org))

Affichage des catégories afin de confirmer d'avoir deux principales catégories, ceci est remarquable en lisant les noms des catégories, ou chaque catéogrie a une relation avec le baseball ou bien l'athéisme:

In [ ]:
# Show the distinct Categories to confirm that we have two principale categories:

print(Df.select('Category').distinct().show(36))

Affichage d'une description du DataFrame, ceci nous permettra d'avoir les différentes informations concenant les differentes variables:

In [ ]:
# Show the different variables of our data Frame
print("The description of the data frame variables:")
print(Df.describe().show())

Affichage des 10 premiers lignes du dataFrame, ceci nous donnera une idée sur notre donnés:

In [ ]:
#Show the first 10 rows : 
print("The first 10 rows:")
print(Df.show(10))

Affichage du nombre des lignes:

In [ ]:
#The number of rows : 
print('The number of mails in the dataset is: {}'.format((Df.count())))

Affichage d'un resumé de notre DataSet:

In [ ]:
Df.summary().show()

<h1>Transformation du texte et clustering</h1>

Dans cette Partie, on a fais la transformation de text et le clustering en utilisant pyspark:

Dans cette partie, on a fait la tokenization des text, ceci consiste a séparer le text en un ensemble des mots. Après cela, on applique une supprision des stopwords ou ceci consiste a supprimer les mots qui n'ont pas vraiement un poids, et qui se repetent dans tout les sujets:

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer,StopWordsRemover
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="Body", outputCol="Body_tokinized")
stopwordsremover = StopWordsRemover(inputCol='Body_tokinized', outputCol='Body_stopwords')
ponctuation = StopWordsRemover(inputCol='Body_stopwords', outputCol='Body_words',stopWords=['?','!',',',';',':','(',')','[' ,']','«','»','–','{','}'])

pipeline = Pipeline(stages=[tokenizer, stopwordsremover,ponctuation])
words = pipeline.fit(Df).transform(Df)

Application du hashingTF qui consiste a hasher chaque mot et lui assister le nombre de répitions, on a spécifier le paramètre 2000 afin de limiter les variables a 2000 mots:

In [ ]:
hashingTF = HashingTF(numFeatures = 2000 ,inputCol="Body_words", outputCol="rawFeatures")
wordsData = hashingTF.transform(words).na.drop('any')

Application de TF-IDF en appuiyant sur le résultat de HASHINGTF:

In [ ]:
idf = IDF().setInputCol("rawFeatures").setOutputCol("features")
idfModel = idf.fit(wordsData)
rescaledData = idfModel.transform(wordsData)
rescaledData.select("features").show()

Dans cette partie on a appliqué l'algorithme Kmeans sur les données résultats de TF-IDF :

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans(featuresCol='features').setK(2).setSeed(1)

# Make predictions
model = kmeans.fit(rescaledData)
model

# Make predictions
predictions = model.transform(rescaledData)

predictions.select("prediction").distinct().count()
predictions.select("Category","prediction")
predictions.summary().show()

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator(featuresCol='rawFeatures')

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

La récupération des labels des prédits par l'algorithme de Kmeans de PySpark:

In [ ]:
import numpy as np
pred = predictions.select('prediction').collect()
resl = np.array([])
for elm in pred:
  resl = np.append(resl, elm[0])



Création d'un vecteur qui regroupe les classes des documents:

In [ ]:
y = np.zeros(1877)
y[1000:] = 1

La récupération des données utilisé pour chaque document dans l'algorithme de Kmeans, ceci est dans le but de réappliquer l'algorithme de Kmeans avec Sickit Learn:

In [ ]:
x = np.array(rescaledData.select('features').collect())
X = np.array([])
for elm in x :
  X = np.append(X,elm[0])

X = X.reshape(1877,2000)

Application du Kmeans avec Sickit learn:

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)


Calcule de NMI pour la classification de PySPark:

In [ ]:
from sklearn.metrics.cluster import normalized_mutual_info_score

normalized_mutual_info_score(resl,y)

Calcule de NMI pour la classification de Sickit Learn:

In [ ]:
normalized_mutual_info_score(kmeans.labels_, y)

On peux remarquer l'implimentation de Pyspark est beaucoup meilleur que celle de SickitLeanr en terme des résultats.

<h1>Implémentation de K-means unidimensionnel</h1>

Dans cette partie, on a implimenté la K-Means unidmensionnel:

On a commencé par l'implimentation de la fonction qui calcule les centroides de chaque cluster, ceci est fait grace a des fonctions sur les RDD:

In [ ]:
def compute_centroids(points,cluster_ids):
  # ( Id_Cluster, Sum of element in that cluster ) 
  sum_by_cluster = cluster_ids.zip(points).groupByKey().mapValues(sum)

  # (Id_Cluster, Number of occurence)
  count_by_cluster_id = cluster_ids.zip(points).groupByKey().mapValues(len)

  # ( Id_Cluster, Mean):
  return count_by_cluster_id.join(sum_by_cluster).map(lambda x : (x[0],x[1][1]/x[1][0])).sortByKey()








Dans la deuxième phase, on a implimenté la fonction assign clusters qui assigne chaque point a un cluster, ceci se base sur la distance carré qui est implimenté dans la fonction squared sitances:

In [ ]:
import numpy as np
def squared_distances(point, centroids):
  return np.sqrt(np.power(centroids-point,2))

In [ ]:
def assign_clusters(points, centroids):
  list_centr = np.array(centroids.map(lambda x: x[1]).collect())
  return points.map(lambda x : np.argmin(squared_distances(x,list_centr)))


Dans cette Partie, on a utilisé les fonctions déja défini, et on a implimenté l'algorithme de Kmeans Unidimensionnel, il prend en paramètres les points et le nombre des clusters, il choisi au hasard k points, et il applique le principe de kmeans jusqu'a la convergence:

In [ ]:
def KmeansUniDim(points,k):
  centroids = np.array(points.collect()[0:k])
  clusters_ids = points.map(lambda x : np.argmin(squared_distances(x,centroids))).collect()
  new_list_id_clusters = []
  #1st itiration:  
  centroids = compute_centroids(points,sc.parallelize(clusters_ids))
  new_cluster_ids = assign_clusters(points, centroids).collect()
  #The stop condition:
  while(not np.array_equal(new_cluster_ids,clusters_ids) ):
    
    clusters_ids = new_cluster_ids
    centroids = compute_centroids(points,sc.parallelize(clusters_ids))
    new_cluster_ids = assign_clusters(points, centroids).collect()
  
  return clusters_ids,centroids





#Implémentation de K-means multidimensionnel
---





Dans cette partie, on s'est basé sur la première implimentation du Kmeans pour pouvoir implimenter le Kmeans multidimensionnel, le meme principe a été utilisé, avec quelque modifications sur les dimensions.

La fonction calculate_sum calcule la somme des dimensions afin de pouvoir calculer la variable sum_by_cluster. 

La fonction squared_distances_multiple est responsable de retourner les distance entre le point et chaque cluster.

In [ ]:
def calculate_sum(X):
  sum = np.zeros(len(X[0]))
  for elm in X:
    sum = np.add(sum,elm)
  return sum.astype(int)



def compute_centroids_multidimensionnel(points,cluster_ids):
  # ( Id_Cluster, Sum of element in that cluster ) 
  sum_by_cluster = cluster_ids.zip(points).groupByKey().map(lambda x : (x[0], list(x[1]))).mapValues(calculate_sum)

  # (Id_Cluster, Number of occurence)
  count_by_cluster_id = cluster_ids.zip(points).groupByKey().mapValues(len)

  # ( Id_Cluster, Mean):
  return count_by_cluster_id.join(sum_by_cluster).map(lambda x : (x[0],x[1][1]/x[1][0])).sortByKey()




def assign_clusters_multidimensionnel(points, centroids):
  list_centr = np.array(centroids.map(lambda x: x[1]).collect())
  return points.map(lambda x : np.argmin(squared_distances_multiple(np.array(x),list_centr)))

def squared_distances_multiple(point, centroids):
    return np.sqrt(np.sum(np.power(centroids-point,2),axis=1))
  


  


Cette fonction s'appuie sur les fonctions déja défini afin de donner une implimentation du kmeans multidimentionnel, l'intialisation est également random selon le nombre des clusters:

In [ ]:
def KmeansMultiDim(points,k):
  deb = np.random.randint(len(points.collect())-k)
  centroids = np.array(points.collect()[deb:deb+k])
  clusters_ids = points.map(lambda x : np.argmin(squared_distances_multiple(x,centroids))).collect()
  new_list_id_clusters = []
  
  #1st itiration:  
  centroids = compute_centroids_multidimensionnel(points,sc.parallelize(clusters_ids))
  new_cluster_ids = assign_clusters_multidimensionnel(points, centroids).collect()
  #The stop condition:
  while(not np.array_equal(new_cluster_ids,clusters_ids) ):
    
    clusters_ids = new_cluster_ids
    centroids = compute_centroids_multidimensionnel(points,sc.parallelize(clusters_ids))
    new_cluster_ids = assign_clusters_multidimensionnel(points, centroids).collect()
  
  return clusters_ids,centroids





In [ ]:
points = sc.parallelize([[1,2,3],[1,3,5],[1,5,9]])
clusters_ids = sc.parallelize([0,1,0]) 

sum_by_cluster = clusters_ids.zip(points).groupByKey().map(lambda x : (x[0], list(x[1]))).mapValues(calculate_sum)

count_by_cluster_id = clusters_ids.zip(points).groupByKey().mapValues(len)

count_by_cluster_id.join(sum_by_cluster).collect()

#Implémentation de Spherical K-Means
---





L'implémentation de SPherical K-Means est également du meme principe que l'implimentation de K-Means multidimension, la seule difference c'est que Spherical Kmeans s'appuiue sur la distance cosianne, ceci est fait a l'aide du package scipy et la fonction spatial, donc SKeamns s'appuie plutot sur le notion de profil

assignation des clusters s'appuie donc sur la fonction distance_cosine qui définit la distance cosianne entre deux individu ( en prendre en compte le cosinus de l'angle entre deux vecteur )

In [ ]:
from scipy import spatial

def assign_clusters_spherical(points, centroids):
  list_centr = np.array(centroids.map(lambda x: x[1]).collect())
  return points.map(lambda x : np.argmax(distance_cosine(np.array(x),list_centr)))

def distance_cosine(X,list_centr):
  distance = np.array([])
  for center in list_centr:
    distance = np.append(distance,1 - spatial.distance.cosine(X,center))
  print('hello')
  return distance


Comme les deux autres, cette fonction représente l'implimentation du sphérical Kmeans, elle s'appuie sur l'assignation des clusters spherical, et la distance cosine, ainsi que le calcule centroids multidimensionnel afin d'implimenter l'algorithme spherical Kmeans.

L'initialisation est random en suivant la valeur k.

In [ ]:
def SKmeansMultiDim(points,k):
  deb = np.random.randint(len(points.collect())-k)
  centroids = np.array(points.collect()[deb:deb+k])
  clusters_ids = points.map(lambda x : np.argmax(distance_cosine(x,centroids))).collect()
  
  new_list_id_clusters = []
  #1st itiration:  
  centroids = compute_centroids_multidimensionnel(points,sc.parallelize(clusters_ids))
  new_cluster_ids = assign_clusters_spherical(points, centroids).collect()
  #The stop condition:
  while(not np.array_equal(new_cluster_ids,clusters_ids) ):
    
    clusters_ids = new_cluster_ids
    centroids = compute_centroids_multidimensionnel(points,sc.parallelize(clusters_ids))
    new_cluster_ids = assign_clusters_spherical(points, centroids).collect()
  
  return clusters_ids,centroids





#Analyser les résultats:

Dans cette section, on analyse les résultats des différents implimentation des algorithme de kmeans:

### 1. Unidimensionnel Kmeans:

On commence par définir 8 points, qui represent 3 clusters, le premier regroupe 1,0,-1 ( proche de 10) , le deuxième -20,-12,-15 ( largement inferieur a zero ) , et le troisième c'est 20,30 ( largement supérieur a 0 );

In [ ]:
points = sc.parallelize([1,-20,20,0,-1,-15,30,-12])
labels, centroids =KmeansUniDim(points,3)

Le résultat est le suivant:

In [ ]:
labels

Les centroids sont les suivant qui sous format de tuple, le premier represente le cluster, et le deuxième le centre de ce cluster:

In [ ]:
centroids.collect()

### 2.SKmeans

Le deuxième exemple consiste a implimenté un Skmeans sur les données, ou cette exemple est constitué de deux cluster, le premier est celui des vecteur avec x positif, et les autres avec x négatifs. 

Notons que plusieurs intialisation peuvent conduire a plusiers résultats, donc il est préférable de relancer l'exécution plusieurs fois pour avoir les meilleurs résultats:

In [ ]:
Multiple_points = sc.parallelize([[1, 1], [2, 2], [3, 3], [-1, 1], [-2, 2], [-3, 3]])

In [ ]:
labels,Centeroids = SKmeansMultiDim(Multiple_points,2)
labels

In [ ]:
Centeroids.collect()

### 3.Multidimensionel kmeans:

Dans cette exemple, on s'intéresse a un cas de 3 dimensions avec 2 cluster

In [ ]:
points = sc.parallelize([[1, 1,1], [2, 2,2], [3, 3,3], [-1, -1,-1], [-2, -2,-2], [-3, -3,-3]])

In [ ]:
labels,Centeroids = KmeansMultiDim(points,2)
labels

In [ ]:
Centeroids.collect()

Dans cette exemple, on compare les résultats avec l'algorithme de Kmeans de sickit-learn:

In [ ]:
X = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3], [-1, -1, -1], [-2, -2, -2], [-3, -3, -3]])
labels_sickit = KMeans(n_clusters=2).fit(X)

In [ ]:
labels_sickit.labels_

On peux remarquer qu'on a exactement le meme résultat que le Kmeans de Sickit-Learn.